# Assignment 2

## Problem 2

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import numpy.random as nr
import sklearn.model_selection as ms
import math
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_json('https://raw.githubusercontent.com/wncc/learners-space/master/Machine%20Learning/MId%20Term%20Assignment/data.json')
data.head()



,0,1,2
0,-3.005382,-1.701191,1
1,-2.908105,-1.699589,1
2,-2.958954,-1.660940,1
3,-2.922840,-1.638722,1
4,-2.924240,-1.508256,1


In [138]:
data[2].value_counts()

1    4679
0    1321
Name: 2, dtype: int64

In [139]:
nr.seed(500)
X = data.iloc[:,:2]
Y = data.iloc[:,2:]
x_train,x_test,y_train,y_test = ms.train_test_split(X,Y, test_size = 0.4)

In [140]:
np.array(x_train).reshape(3600,2)


array([[-1.49586208,  0.29523945],
       [-0.10238082,  0.07442977],
       [-1.04704195,  1.22658971],
       ...,
       [-2.86649872,  0.70749098],
       [ 0.93180197, -1.55120221],
       [-2.02902349, -0.04887253]])

In [141]:
np.array(y_train).reshape(3600,1)

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [1]], dtype=int64)

In [142]:
A0 = np.array(pow(x_train[0],4)).reshape(1, x_train.shape[0])
A1 = np.array(pow(x_train[0],3)*x_train[1]).reshape(1, x_train.shape[0])
A2 = np.array(pow(x_train[0],2)*pow(x_train[1],2)).reshape(1, x_train.shape[0])
A3 = np.array(pow(x_train[0],1)*pow(x_train[1],3)).reshape(1, x_train.shape[0])
A4 = np.array(pow(x_train[0],4) ).reshape(1, x_train.shape[0])
A = np.vstack((A0, A1, A2, A3, A4))
A = A.T
print(A)

[[ 5.00686878e+00 -9.88209555e-01  1.95043683e-01 -3.84959222e-02
   5.00686878e+00]
 [ 1.09868822e-04 -7.98734706e-05  5.80671676e-05 -4.22142162e-05
   1.09868822e-04]
 [ 1.20186678e+00 -1.40796406e+00  1.64940309e+00 -1.93224432e+00
   1.20186678e+00]
 ...
 [ 6.75160474e+01 -1.66638813e+01  4.11287318e+00 -1.01511319e+00
   6.75160474e+01]
 [ 7.53866579e-01 -1.25498716e+00  2.08921952e+00 -3.47799431e+00
   7.53866579e-01]
 [ 1.69491646e+01  4.08249848e-01  9.83340138e-03  2.36854424e-04
   1.69491646e+01]]


In [143]:
coef = np.array([0.45, 0.001, 0.001, 0.001, 0.001])

In [144]:
y=y_train
for i in range (250):
    coef = coef.reshape(5,1) - 0.2*(np.dot(np.array((np.array(1/(1+
           (np.exp(-(np.dot(A,coef.reshape(5,1)))))))).reshape(y.shape[0],1) - y).reshape([1,y.shape[0]]), A)/y.shape[0]).reshape(5,1)
    
print(coef)
y_train=y

[[ 1.24910007]
 [-0.07761834]
 [-1.30610338]
 [ 0.01043576]
 [ 0.80010007]]


In [171]:
def h(A, coef):
  return np.array(1/(1+(np.exp(-f(A,coef)))) )
def f(A, coef):
  fun = np.dot(A,coef.reshape(5,1))
  return fun
def l(y, A, coef): 
  return (-np.dot(y.T, np.log(h(A, coef))) - np.dot((1-y).T ,np.log(1-h(A, coef))) )/y.shape[0]
def g(A, coef, y):
  return np.dot(np.array(h(A, coef).reshape(y.shape[0],1) - y).reshape([1,y.shape[0]]), A)/y.shape[0]
def gradesc(A, y, coef,L):      
  for i in range (250):
    coef = coef.reshape(5,1) - L*g(A, coef, y).reshape(5,1)
  return coef 

def predict(x_test, coef = gradesc(A,y_train, coef, 0.01) ):
  A0 = np.array(pow(x_test[0],4)).reshape(1, x_test.shape[0])
  A1 = np.array(pow(x_test[0],3)*x_test[1]).reshape(1, x_test.shape[0])
  A2 = np.array(pow(x_test[0],2)*pow(x_test[1],2)).reshape(1, x_test.shape[0])
  A3 = np.array(pow(x_test[0],1)*pow(x_test[1],3)).reshape(1, x_test.shape[0])
  A4 = np.array(pow(x_test[0],4) ).reshape(1, x_test.shape[0])
  A_ = np.vstack((A0, A1, A2, A3, A4))
  A_ = A_.T
  print('Mean Loss :' ,np.sum(l(y_test,A_,coef)))
  return h(A_, coef)





probs = predict(x_test)
print(probs)

Mean Loss : nan
[[0.69577536]
 [1.        ]
 [1.        ]
 ...
 [0.5810489 ]
 [0.99999528]
 [0.21463553]]


<ipython-input-171-0c3c5c756380>:7: RuntimeWarning: divide by zero encountered in log
  return (-np.dot(y.T, np.log(h(A, coef))) - np.dot((1-y).T ,np.log(1-h(A, coef))) )/y.shape[0]


In [183]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:]])
y_pred = score_model(probs, 0.1)
pred_df = pd.DataFrame(y_pred)
print('Predicted values')
print(pred_df.value_counts())

Predicted values
1    2400
dtype: int64


In [184]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:]])
y_pred = score_model(probs, 0.5315)
pred_df = pd.DataFrame(y_pred)
print('Predicted values')
print(pred_df.value_counts())
print('Actual values')
y_test.value_counts()

Predicted values
1    1774
0     626
dtype: int64
Actual values


2
1    1885
0     515
dtype: int64

In [185]:
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[1,1] + '             %5d' % conf[1,0])
    print('Actual negative    %6d' % conf[0,1] + '             %5d' % conf[0,0])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][1] + '        %6d' % metrics[3][0])
    print('Precision  %6.2f' % metrics[0][1] + '        %6.2f' % metrics[0][0])
    print('Recall     %6.2f' % metrics[1][1] + '        %6.2f' % metrics[1][0])
    print('F1         %6.2f' % metrics[2][1] + '        %6.2f' % metrics[2][0])


    
print_metrics(y_test, y_pred)

                 Confusion matrix
                 Score positive    Score negative
Actual positive      1737               148
Actual negative        37               478

Accuracy  0.92
 
           Positive      Negative
Num case     1885           515
Precision    0.98          0.76
Recall       0.92          0.93
F1           0.95          0.84
